In [1]:
import cv2
import numpy as np
import glob
import os
import depthai as dai
import numpy as np

In [2]:
NO_OF_IMAGES = 20
IMAGES_FOLDER = 'images_folder'

In [3]:
def getFrame(queue):
    frame = queue.get()
    return frame.getCvFrame()

def getMonoCamera(pipeline, isLeftCamera):
    monoCamera = pipeline.createMonoCamera()
    monoCamera.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
    if isLeftCamera:
        monoCamera.setBoardSocket(dai.CameraBoardSocket.LEFT)
    else:
        monoCamera.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    return monoCamera

def getColorCamera(pipeline):
    colorCamera = pipeline.createColorCamera()
    colorCamera.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    return colorCamera

def create_images_folders(imageType):
    current_directory = os.getcwd()
    sub_images_directory = os.path.join(current_directory, IMAGES_FOLDER+'/'+imageType)
    if not os.path.exists(sub_images_directory):
        os.makedirs(sub_images_directory)

# Create Images Folders
create_images_folders('left')
create_images_folders('right')
create_images_folders('rgb')

In [4]:
pipeline = dai.Pipeline()

In [ ]:


# Capturing Images from Left Camera
leftMonoCamera = getMonoCamera(pipeline,isLeftCamera=True)

# Creating output
xLeftLinkout = pipeline.createXLinkOut()
xLeftLinkout.setStreamName('left')
leftMonoCamera.out.link(xLeftLinkout.input)

with dai.Device(pipeline) as device:
    queue_left = device.getOutputQueue(name='left', maxSize=4, blocking=False)
    for i in range(NO_OF_IMAGES):
        leftFrame = getFrame(queue_left)
        cv2.imshow('left', leftFrame)
        cv2.imwrite(f"{IMAGES_FOLDER}/left/left_image_{i+1}.png", leftFrame)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
    print(f"Using Left Camera, {NO_OF_IMAGES} images captured")

# Capturing Images from Right Camera
rightMonoCamera = getMonoCamera(pipeline,isLeftCamera=False)

# Creating output
xRightLinkout = pipeline.createXLinkOut()
xRightLinkout.setStreamName('right')
rightMonoCamera.out.link(xRightLinkout.input)

with dai.Device(pipeline) as device:
    queue_right = device.getOutputQueue(name='right', maxSize=4, blocking=False)
    for i in range(NO_OF_IMAGES):
        rightFrame = getFrame(queue_right)
        cv2.imshow('right', rightFrame)
        cv2.imwrite(f"{IMAGES_FOLDER}/right/right_image_{i+1}.png", rightFrame)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
    print(f"Using Right Camera, {NO_OF_IMAGES} images captured")

# Capturing Images from RGB Camera
colorCamera = getColorCamera(pipeline)

# Creating output
xRGBout = pipeline.createXLinkOut()
xRGBout.setStreamName("rgb")
colorCamera.video.link(xRGBout.input)
with dai.Device(pipeline) as device:
    queue_rgb = device.getOutputQueue(name='rgb', maxSize=4, blocking=False)
    for i in range(NO_OF_IMAGES):
        rgbFrame = getFrame(queue_rgb)
        rgbFrameResized = cv.resize(rgbFrame, (960, 540))
        cv2.imshow('rgb', rgbFrameResized)
        cv2.imwrite(f"{IMAGES_FOLDER}/rgb/rgb_image_{i+1}.png", rgbFrame)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
    print(f"Using RGB Camera, {NO_OF_IMAGES} images captured")

In [ ]:
def perform_calibration(images,imageType):
    objectPoints = []
    imagePoints = []
    
    objp = np.zeros((1, 6*9, 3), np.float32)
    objp[0,:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
    imagesWithCorner = []
    for each_image in images:
        img = cv2.imread(each_image)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        retval, corners = cv2.findChessboardCorners(gray, (9,6), None)
        if retval == True:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), termination_criteria)
            imgpoints.append(corners2)
            img = cv2.drawChessboardCorners(img, (9,6), corners2, retval)
            cv2.imshow('img', img)
            cv2.imwrite({each_image.split('.')[0]}+"_corners.png", img)
            cv2.waitKey(1000)
            cv2.destroyAllWindows()
            imagesWithCorner.append(each_image)
    cv2.destroyAllWindows()
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, gray.shape[::-1], None, None)

    for each_image in images:
        img = cv2.imread(each_image)
        dst = undistort_image(img, mtx, dist)
        # Save the undistorted image
        cv2.imwrite(f"{each_image.split('.')[0]}_result.png", dst)
    
    # Saving camera matrix and distortion coefficients
    np.savetxt(IMAGES_FOLDER+"/"+imageType+"/camera_matrix.txt",mtx)
    np.savetxt(IMAGES_FOLDER+"/"+imageType+"/distortion_vector.txt",dist)
    np.savetxt(IMAGES_FOLDER+"/"+imageType+"/rotation_vectors.txt",np.array(rvecs).reshape(-1, 3))
    np.savetxt(IMAGES_FOLDER+"/"+imageType+"/translation_vectors.txt",np.array(tvecs).reshape(-1, 3))
    print("All camera martix and distortion coefficients are saved")


color_images = glob.glob(IMAGES_FOLDER+'/rgb/*.png')
left_images = glob.glob(IMAGES_FOLDER+'/left/*.png')
right_images = glob.glob(IMAGES_FOLDER+'/right/*.png')

In [ ]:
def load_camera_data(camera_matrix_file, distortion_vector_file):
    camera_matrix = []
    extrinsic_matrix = []
    with open(camera_matrix_file, 'r') as f:
        for line in f:
            camera_matrix.append([float(num) for num in line.split(' ')])
    with open(distortion_vector_file, 'r') as f:
        for line in f:
            extrinsic_matrix.append([float(num) for num in line.split(' ')])
    return np.array(camera_matrix), np.array(extrinsic_matrix)

def load_stereo_calibration_data():
    camera_matrix, extrinsic_matrix = load_camera_data(IMAGES_FOLDER+'/left/camera_matrix.txt', IMAGES_FOLDER+'/left/distortion_vector.txt')
    
    return camera_matrix, extrinsic_matrix

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def main():
    # Load calibration data
    camera_matrix_left, dist_coeffs_left, camera_matrix_right, dist_coeffs_right, R, T = load_stereo_calibration_data()
    baseline = 0.1

    cap_left = cv2.VideoCapture(0)  # Adjust camera index
    cap_right = cv2.VideoCapture(1)  # Adjust camera index

    tracker = setup_tracker()

    while True:
        ret_left, frame_left = cap_left.read()
        ret_right, frame_right = cap_right.read()

        success, bbox = tracker.update(frame_left)
        if success:
            # Drawing bounding box
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame_left, p1, p2, (255,0,0), 2, 1)
            
            # Estimate dimensions based on the stereo images
            width, height, depth = estimate_dimensions(frame_left, frame_right, camera_matrix_left, dist_coeffs_left, camera_matrix_right, dist_coeffs_right, baseline)
            print(f"Dimensions: Width={width}m, Height={height}m, Depth={depth}m")

        # Show the left frame with bounding box and dimensions
        cv2.imshow("Stereo Object Tracking", frame_left)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_left.release()
    cap_right.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


NameError: name 'load_stereo_calibration_data' is not defined